In [ ]:
conda create -n python=3.10

In [1]:
pip install open3d

  Using cached werkzeug-3.1.3-py3-none-any.whl.metadata (3.7 kB)
  Using cached flask-3.1.0-py3-none-any.whl.metadata (2.7 kB)
  Using cached nbformat-5.10.4-py3-none-any.whl.metadata (3.6 kB)
  Using cached addict-2.4.0-py3-none-any.whl.metadata (1.0 kB)
  Using cached PyYAML-6.0.2-cp310-cp310-macosx_11_0_arm64.whl.metadata (2.1 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached flask-3.0.3-py3-none-any.whl.metadata (3.2 kB)
  Using cached werkzeug-3.0.6-py3-none-any.whl.metadata (3.7 kB)
  Using cached retrying-1.3.4-py3-none-any.whl.metadata (6.9 kB)
  Using cached itsdangerous-2.2.0-py3-none-any.whl.metadata (1.9 kB)
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached kiwisolver-1.4.8-cp310-cp310-macosx_11_0_arm64.whl.metadata (6.2 kB)
  Using cached scipy-1.15.2-cp310-cp310-macosx_12_0_arm64.whl.metadata (61 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.1/103.1 MB 509.8 kB/s eta 0:00:0000:0100:06
   ━━━━━━━━━━━━━━━━━━━━━

In [11]:
pip install open3d==0.19.0

  Using cached open3d-0.19.0-cp310-cp310-macosx_10_15_universal2.whl.metadata (4.2 kB)
Using cached open3d-0.19.0-cp310-cp310-macosx_10_15_universal2.whl (103.1 MB)
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import open3d as o3d
import numpy as np
def align_face_to_axis(pcd, target_axis=np.array([0, 0, 1])):
    # Center the point cloud at the origin
    center = pcd.get_center()
    pcd.translate(-center)

    # Estimate normals
    pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))

    # Compute average face direction
    normals = np.asarray(pcd.normals)
    face_direction = np.mean(normals, axis=0)
    face_direction /= np.linalg.norm(face_direction)

    # Compute rotation to target axis
    target_axis = target_axis / np.linalg.norm(target_axis)
    rotation_axis = np.cross(face_direction, target_axis)

    if np.linalg.norm(rotation_axis) < 1e-6:
        print("Face already aligned with target axis.")
        return pcd

    rotation_axis /= np.linalg.norm(rotation_axis)
    angle = np.arccos(np.clip(np.dot(face_direction, target_axis), -1.0, 1.0))
    R = o3d.geometry.get_rotation_matrix_from_axis_angle(rotation_axis * angle)

    # Apply rotation around the origin (already centered)
    pcd.rotate(R, center=(0, 0, 0))
    return pcd



import numpy as np
def find_nose_tip(pcd, axis=np.array([0, 0, 1])):
    points = np.asarray(pcd.points)
    # Project all points onto the axis (e.g., Z-axis)
    projections = points @ axis
    max_index = np.argmax(projections)
    nose_tip = points[max_index]
    print(nose_tip)
    print(f"Nose tip coordinates: {nose_tip}")
    return nose_tip

def find_nose_tip_refined(pcd, axis=np.array([0, 0, 1]), k=30):
    points = np.asarray(pcd.points)
    normals = np.asarray(pcd.normals)

    # Project onto the given axis (e.g., Z)
    projections = points @ axis
    max_index = np.argmax(projections)
    rough_tip = points[max_index]

    # Use KDTree to search nearby points for refinement
    pcd_tree = o3d.geometry.KDTreeFlann(pcd)
    [_, idxs, _] = pcd_tree.search_knn_vector_3d(rough_tip, k)
    candidate_points = points[idxs]
    candidate_normals = normals[idxs]

    # Choose point with normal most aligned with axis
    dot_products = candidate_normals @ axis
    best_index = np.argmax(dot_products)
    nose_tip = candidate_points[best_index]

    print(f"Refined nose tip coordinates: {nose_tip}")
    return nose_tip


# Load your point cloud
pcd = o3d.io.read_point_cloud("someone3.ply")
ta=np.array([0, 0, 1])
aligned_pcd = align_face_to_axis(pcd, target_axis=ta)

# Find the nose tip

nose_tip = find_nose_tip(aligned_pcd,ta)

# Add a small sphere at the nose tip for visualization
nose_sphere = o3d.geometry.TriangleMesh.create_sphere(radius=1)
nose_sphere.translate(nose_tip)
nose_sphere.paint_uniform_color([1, 0, 0])  # Red

# Visualize the result
coord_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=100)
# Create a plane perpendicular to Z-axis at the nose tip
plane_size = 20  # You can adjust this for visibility
plane = o3d.geometry.TriangleMesh.create_box(width=plane_size, height=plane_size, depth=1e-4)
plane.translate(nose_tip - np.array([plane_size/2, plane_size/2, 0]))
plane.paint_uniform_color([0.2, 0.8, 1.0])  # Light blue

# Visualize everything
o3d.visualization.draw_geometries([aligned_pcd, nose_sphere, coord_frame])
